In [48]:
from qiskit.circuit.library import RYGate, RZGate, CXGate, CRZGate
from qiskit.quantum_info import SparsePauliOp

import numpy as np
import scipy


In [80]:
alpha = 2 * np.arctan(np.sqrt(0.5))
beta = 0.5 * np.pi
gamma = -1.5 * np.pi

ry_alpha = np.kron(RYGate(alpha).to_matrix(), np.eye(2))
ry_beta = np.kron(np.eye(2), RYGate(beta).to_matrix())
# rz_gamma = np.kron(np.eye(2), RZGate(gamma).to_matrix())
zero_not = CXGate(ctrl_state=0).to_matrix()
crz = CRZGate(gamma, ctrl_state=0).to_matrix()

print(crz)
# qc.crz(gamma, control_qubit=0, target_qubit=1)


np.set_printoptions(precision=3)
# print(ry_alpha)
# print(ry_beta)
# print(rz_gamma)

[[-0.707+0.707j  0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     1.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j    -0.707-0.707j  0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     1.   +0.j   ]]


In [41]:
step1 = (ry_alpha @ ry_beta) @ np.array([1, 0, 0, 0])
# step2 = zero_not @ step1
step2 = step1 @ zero_not
step3 = crz @ step2

print(step1)
print(step2)
print(step3)

[0.577+0.j 0.577+0.j 0.408+0.j 0.408+0.j]
[0.408+0.j 0.577+0.j 0.577+0.j 0.408+0.j]
[-0.289+0.289j  0.577+0.j    -0.408-0.408j  0.408+0.j   ]


In [102]:
np.set_printoptions(precision=5)
def get_hamilt_op(N, lambd):
    hlist = []
    coeffs = []
    
    for i in range(N - 1):
        text = "I" * i + "ZZ" + "I" * (N - i - 2)
        hlist.append(text)
        coeffs.append(1)

    if lambd:
        for i in range(N):
            text = "I" * i + "X" + "I" * (N - i - 1)
            hlist.append(text)
            coeffs.append(lambd)

    return SparsePauliOp(hlist, coeffs=coeffs)

ham = get_hamilt_op(2, 0.5).to_matrix()
print(ham)

evop1 = scipy.linalg.expm(ham * 1j)
evop2 = scipy.linalg.expm(ham * -2j)


avt1 = np.array([1, 0, 0, 0])# / np.sqrt(2)
stage1 = evop1 @ avt1
print(stage1, np.abs(stage1), np.abs(stage1)**2)
stage2 = evop2 @ stage1

# print(stage2, np.abs(stage2), np.abs(stage2)**2)

# print(evop2 @ np.array([1, 0, 0, 0]), np.abs(evop2 @ np.array([1, 0, 0, 0])), np.abs(evop2 @ np.array([1, 0, 0, 0]))**2)

# (evop2) @ np.array([1, 1, 1, 1]) * 0.5


[[ 1. +0.j  0.5+0.j  0.5+0.j  0. +0.j]
 [ 0.5+0.j -1. +0.j  0. +0.j  0.5+0.j]
 [ 0.5+0.j  0. +0.j -1. +0.j  0.5+0.j]
 [ 0. +0.j  0.5+0.j  0.5+0.j  1. +0.j]]
[ 3.48123e-01+0.76996j -4.33681e-18+0.34923j -8.67362e-18+0.34923j
 -1.92179e-01-0.07151j] [0.845   0.34923 0.34923 0.20505] [0.71403 0.12196 0.12196 0.04205]
